<h1>Die Convolution und Pooling Operationen</h1><br>
Convolutional Neural Networks dienen der Arbeit mit Bildern. Ihre Genauigkeit übertrifft denen normaler NNs. Das liegt daran, dass bevor die Bilder in dicht vernetzte Schichten (<code>Dense</code>) eingespeist werden, werden sie durch zwei andere Arten von Schichten geschickt.<br><br>
<b>Die Convolution-Operation</b><br>
Abstrakt gesagt ist der Unteschied zwischen normalen NNs und CNNs, dass CNNs lokale Muster lernen, während NNs sich auf gloable Muster beschränken müssen. Dabei lernen die Convolutional Schichten die lokalen Muster. Ist eine Muster an einer Stelle im Bild gelernt, kann es überall wiedererkannt werden. Mehrere Convolutional Schichten hintereinander können komplexere Muster im Bild erkennen und somit etwas anhand komplexer Konzepte im Bild erkennen. Das passiert, indem die einfachen Muster der ersten Convolution-Schicht, wie "vertikaler Strich" oder "Strich von oben links nach unten rechts", sich in darauffolgenden Schichten zu komplexeren Mustern zusammensetzen, wie "Ohr" oder "Auge" beim Erkennen menschlicher Gesichter.<br><br><img src="./imgs/cnn_cat.png"><br><br>
Der Input in ein CNN sind Bilder mit den Dimensionen <code>(height, width, channels)</code>. Die Channels beschreiben z.B. Farbanteile wie rot, grün, blau (RGB). Im Falle von RGB wären es also drei Channel. Im Falle eines schwarz-weißen Bildes wäre es 1, die Graustufen. Die Convolution extrahiert darauf kleinere Teile des Bildes, meist der Größe (3, 3) oder (5, 5). Über diese wird Pixel für Pixel ein Filter geschoben, der kleine Muster, wie z.B. "querer Strich" kodiert. Der Rückgabewert der Schicht ist dann ein Tensor der Dimension <code>(height - k, width - k, Anzahl Filter)</code>. k sind ein oder zwei Pixel am Rand, die nicht mit einbezogen werden können, weil die Filter sonst teilweise nicht im Bild liegen würden. Jeder Filter gibt eine sogenannte Response Map (<code>(height - k, width - k, 1)</code>) zurück, die zeigt, welche Teile des Bilder am ehesten dem Filter entsprechen.<br><br><img src="./imgs/cnn_filter.png"><br><br>
Dabei wird zuerst der kleine Teil extrahiert. Das "filtern" erfolgt über Matrixoperation mit einem Kernel. Der Kernel kodiert das Konzept, indem er die Bedeutung der Pixel um den Mittelpixel gewichtet. Die Berechnung erfolgt dann mittels der Matrixmultiplikation zwischen Ausschnitt und Kernel/Filter für jeden Kernel (Anzahl Kernel -> Output Depth), Dimension <code>(1, 1, Anzahl Filter)</code>. Dann wird der Ausschnitt untersucht, der sich einen Pixel weiter befindet, wiederholt, usw. Die 1 * 1 Ausschnitte werden dann woeder zur Feature Map zusammengesetzt für jeden der Kernel.<br><br><img src="./imgs/cnn_feature_maps.png"><br><br>
Will man eine Feature Map als Output, die die gleichen Dimensionen hat wie die des Inputs, kann man dies über das Padding steuern. Padding fügt dem Bild am Rand genügend Pixel hinzu, sodass der Ausschnitt niemals außerhalb des Bildes liegen kann. Außerdem beeinflusst das Striding die Größe des Outputs. Striding beschreibt, um wie viele Pixel der Ausschnitt springt. Allerdings wird selten ein Striding != 1 genutzt.<br><br>
<b>Die Max-Pooling-Operation</b><br><br>
Diese Schichten werden genutzt, um ein Bild aggressiv zu verkleinern. Ähnlich den Convolution-Schichten entziehen sie dem Bild kleinere Bilder. Meistens bestehen diese kleineren Bilder aus 2 * 2-Ausschnitten mit einem Stride von 2, also existiert keine Überlappung. Aus diesen Ausschnitten wird eine der Maximalwert entnommen und in die nächste Schicht eingespeist. Aber warum sollte das getan werden? Zum einen werden dadurch die Muster "gepresst", sodass Muster auf einem höheren Level gelernt werden können und zum anderen bedürften das Modell sonst zu viele Parameter, was zu Overfitting führt.

In [29]:
# Aus Datensatz einen kleineren zum üben erstellen
import os, shutil

# create new directory for downsized dataset
org_data_dir = '/home/dominik/Documents/Datasets/cats_and_doggos/train'
small_data_dir = '/home/dominik/Documents/Datasets/cats_and_doggos/small'
if not os.path.isdir(small_data_dir):
    os.mkdir(small_data_dir)
    print('Small noch nicht vorhanden.')
else:
    print('Small vorhanden.')

# directories train, validation and test
train_dir = os.path.join(small_data_dir, 'train')
val_dir = os.path.join(small_data_dir, 'val')
test_dir = os.path.join(small_data_dir, 'test')

if not os.path.isdir(train_dir):
    os.mkdir(train_dir)
    os.mkdir(val_dir)
    os.mkdir(test_dir)
    print('Sets noch nicht vorhanden.')
else:
    print('Sets vorhanden.')

train_cats_dir = os.path.join(train_dir, 'cat')
val_cats_dir = os.path.join(val_dir, 'cat')
test_cats_dir = os.path.join(test_dir, 'cat')

train_dogs_dir = os.path.join(train_dir, 'dog')
val_dogs_dir = os.path.join(val_dir, 'dog')
test_dogs_dir = os.path.join(test_dir, 'dog')

if not os.path.isdir(train_cats_dir):
    print('Unterteilungen noch nicht vorhanden.')
    os.mkdir(train_cats_dir)
    os.mkdir(val_cats_dir)
    os.mkdir(test_cats_dir)

    os.mkdir(train_dogs_dir)
    os.mkdir(val_dogs_dir)
    os.mkdir(test_dogs_dir)
else:
    print('Unterteilungen bereits vorhanden.')

print('\n')
if not os.path.exists(os.path.join(train_cats_dir, 'cat.28.jpg')):
    set_ranges = {
        'train': range(1000),
        'val': range(1000, 1500),
        'test': range(1500, 2000)
    }
    
    for animal in ['dog', 'cat']:
        for set in ['train', 'val', 'test']:
            
            file_names = ['{0}.{1}.jpg'.format(animal, i) for i in set_ranges[set]]
            print(file_names[28])
            
            for file_name in file_names:
                src = os.path.join(org_data_dir, file_name)
                dst = os.path.join(small_data_dir, set, animal, file_name)
                shutil.copyfile(src, dst)


Small noch nicht vorhanden.
Sets noch nicht vorhanden.
Unterteilungen noch nicht vorhanden.


dog.28.jpg
dog.1028.jpg
dog.1528.jpg
cat.28.jpg
cat.1028.jpg
cat.1528.jpg
